In [9]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, LeakyReLU, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score
def parse_yolo_label(label_path):
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    # Initialize an empty list to store the parsed labels
    labels = []
    
    for line in lines:
        # Split the line into individual components (e.g., class_id, x_center, y_center, width, height)
        parts = line.strip().split(' ')
        
        # Assuming the label format is class_id x_center y_center width height
        class_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # Append the parsed label to the list of labels
        labels.append([class_id, x_center, y_center, width, height])
    
    return labels

def yolov7(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    x = Conv2D(64, 3, strides=(1, 1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(1024, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(1024, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same')(x)

    x = Conv2D(1024, 3, strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(30, 1, strides=(1, 1), padding='same')(x)
    predictions = Reshape((-1,))(x)

    model = Model(inputs=inputs, outputs=predictions)
    return model

def parse_yolo_label(label_path):
    pass

train_img_dir = "/home/my/Downloads/collegeproject-master/train/images"
train_label_dir = "/home/my/Downloads/collegeproject-master/train/labels"

def load_data(image_dir, label_dir):
    image_paths = sorted(os.listdir(image_dir))
    label_paths = sorted(os.listdir(label_dir))
    data = []
    for image_name, label_name in zip(image_paths, label_paths):
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, label_name)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue
        image = cv2.resize(image, (416, 416))
        image = image.astype(np.float32)
        label = parse_yolo_label(label_path)
        if label is None:
            print(f"Failed to parse label: {label_path}")
            continue
        data.append((image, label))
    return data


train_data = load_data(train_img_dir, train_label_dir)

BATCH_SIZE = 32
IMAGE_SIZE = (416, 416)
NUM_CLASSES = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 50

class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, batch_size, image_size=(416, 416), num_classes=1):
        self.data = data
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = num_classes
        self.num_samples = len(data)

    def __len__(self):
        return int(np.ceil(self.num_samples / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_data = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = []
        batch_labels = []
        for image, label in batch_data:
            processed_image = image
            batch_images.append(processed_image)
            batch_labels.append(label)
        return np.array(batch_images), np.array(batch_labels)

train_generator = CustomDataGenerator(train_data, BATCH_SIZE, IMAGE_SIZE, NUM_CLASSES)

model = yolov7(input_shape=(*IMAGE_SIZE, 3), num_classes=NUM_CLASSES)
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='mse')

model.fit(train_generator, epochs=NUM_EPOCHS)

model.save("nasal_fracture_detector.h5")


Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-10-_png_jpg.rf.1817d2126d7acaf2c4c4e61397d9aa34.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-10-_png_jpg.rf.934f337abf20df54524df2df0fe5b2eb.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-10-_png_jpg.rf.fc9279ba0d92cc49651863e50bd5efff.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-100-_png_jpg.rf.aca46f114b32510187a7b6c50420c853.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-100-_png_jpg.rf.b539efed756946d2ef59558ee0b51f08.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-100-_png_jpg.rf.c60bbf39ad664613564cd6e24cd158c2.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-102-_png_jpg.rf.2e7fc9eb79072805e2caaa5626cf90b0.txt
Failed to parse label: /home/my/Downloads/collegeproject-master/train/labels/a-

ValueError: Expected input data to be non-empty.